#Imports

In [ ]:
%%time

from IPython.display import clear_output

! pip install sentence_transformers==2.2.2

! pip install -qq -U langchain
! pip install -U langchain-community
! pip install -qq -U tiktoken
! pip install -qq -U pypdf
! pip install -qq -U faiss-gpu
! pip install -qq -U InstructorEmbedding

! pip install -qq -U transformers
! pip install -qq -U accelerate
! pip install -qq -U bitsandbytes

! pip install unstructured
! pip install rapidocr-onnxruntime

clear_output()

CPU times: user 1.26 s, sys: 206 ms, total: 1.47 s
Wall time: 3min 6s


In [ ]:

%%time

import warnings
warnings.filterwarnings("ignore")

import os
import glob
import textwrap
import time
from tqdm import tqdm

import langchain
from langchain.document_loaders import PyPDFLoader, DirectoryLoader, UnstructuredMarkdownLoader, TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter, MarkdownHeaderTextSplitter
from langchain import PromptTemplate, LLMChain
from langchain.vectorstores import FAISS

from langchain.llms import HuggingFacePipeline
from langchain.embeddings import HuggingFaceInstructEmbeddings

from langchain.chains import RetrievalQA
from langchain_community.retrievers import TFIDFRetriever
from langchain.retrievers import EnsembleRetriever

import torch
import transformers
from transformers import (
    AutoTokenizer, AutoModelForCausalLM,
    BitsAndBytesConfig,
    pipeline
)



CPU times: user 9.36 s, sys: 1.34 s, total: 10.7 s
Wall time: 20.9 s


# Chunks

In [ ]:
!unzip -o full_doc.zip
clear_output()

In [ ]:
from langchain_community.document_loaders import UnstructuredMarkdownLoader

In [ ]:
loader = DirectoryLoader(
    'full_doc/',
    glob="./*.md",
    loader_cls=TextLoader,
    show_progress=True,
    use_multithreading=True,
)
documents = loader.load()


# loader = DirectoryLoader(
#     'full_doc/',
#     glob="./*.md",
#     loader_cls=UnstructuredMarkdownLoader,
#     show_progress=True,
#     use_multithreading=True,
# )


# documents = loader.load()

# documents = []
# for file in tqdm(os.listdir('full_doc/')):
#     path = 'full_doc/' + file
#     if file[-2:] == 'md':
#         loader = UnstructuredMarkdownLoader(
#             path,
#             glob="./*.md",
#             loader_cls=TextLoader,
#             show_progress=True,
#             use_multithreading=True,
#         )


#         doc = loader.load()
#         documents.append(doc)



# loader = DirectoryLoader(
#     '.',
#     glob="*.pdf",
#     loader_cls=PyPDFLoader,
#     show_progress=True,
#     use_multithreading=True,
# )

# documents = loader.load()

100%|██████████| 386/386 [00:00<00:00, 3210.99it/s]


In [ ]:
print(f'We have {len(documents)} pages in total')

We have 386 pages in total


In [ ]:
# for doc in documents:

#         print(doc.page_content)

In [ ]:
# for page in range(len(documents)):
#     page_content = documents[page].page_content

#     # if 'ак работают платеж' in page_content:
#     #     print(page_content)
#     page_content = page_content.replace('help/assets/images/', 'https://www.rustore.ru/help/assets/images/')
#     documents[page].page_content = page_content

In [ ]:
headers_to_split_on = [
    ("#", "Header 1"),
    ("##", "Header 2"),
    # ("###", "Header 3"),
]

markdown_splitter = MarkdownHeaderTextSplitter(headers_to_split_on=headers_to_split_on,strip_headers=True)

all_splits = []

for doc in documents:
    md_header_splits = markdown_splitter.split_text(doc.page_content)
    all_splits.extend(md_header_splits)
len(all_splits)

2082

In [ ]:
# one_big_split = all_splits[0].copy()
# for split in all_splits[1:]:
#     one_big_split.page_content += '\n' + split.page_content

In [ ]:
# one_big_split

In [ ]:
# all_splits = [split for split in all_splits if len(split.page_content) > 70]

In [ ]:
# len(, len(texts)

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1500,
    chunk_overlap = 400
)

texts = text_splitter.split_documents(all_splits)
# texts = [text for text in texts if len(text.page_content) > 200]
# texts = text_splitter.split_documents([one_big_split])
texts = [text for text in texts if len(text.page_content) > 200]

for i, text in enumerate(texts):
    texts[i].page_content = ' '.join([m for k, m in text.metadata.items()])+ ' ' + texts[i].page_content

In [ ]:
len(texts)

3461

In [ ]:
# for doc in documents:
#     # if 'тистика загрузки верс' in doc.page_content:
#     # if 'просмотреть статистику загрузки версий?' in doc.page_content:
#     if 'ак обновить приложения в RuSt' in doc.page_content:
#         print(doc.page_content)

In [ ]:
# for text in texts:
#     if 'просмотреть статистику загрузки ' in text.page_content:
#         print(text.page_content)

In [ ]:
# import pickle
# with open('md_splits.pickle', 'wb') as f:
#     pickle.dump(all_splits, f)

# Embeddings

In [ ]:
class CFG:
    # LLMs
    model_name = 'distilbert/distilgpt2' # wizardlm, llama2-7b-chat, llama2-13b-chat, mistral-7B
    temperature = 1e-4 # 0
    top_p = 0.9
    repetition_penalty = 1.15

    # splitting
    # split_chunk_size = 1200
    # split_overlap = 200

    # embeddings
    embeddings_model_repo = 'sentence-transformers/all-MiniLM-L6-v2' # 'cointegrated/LaBSE-en-ru' # # #  # 'DeepPavlov/rubert-base-cased-sentence' #  #'cointegrated/LaBSE-en-ru' #
    # embeddings_model_repo =  'intfloat/multilingual-e5-large-instruct' # # 'DeepPavlov/rubert-base-cased-sentence' # 'Den4ikAI/sbert_large_mt_ru_retriever' #'sentence-transformers/all-MiniLM-L6-v2' # 'all-mpnet-base-v2' # 'all-mpnet-base-v2' # 'sentence-transformers/all-MiniLM-L6-v2'

    # similar passages
    k = 4

    # paths
    PDFs_path = 'docs/'
    # PDFs_path = '/content'
    Embeddings_path =  'faiss-hp'
    Output_folder = 'doc_vectordb'

In [ ]:
%%time


embeddings = HuggingFaceInstructEmbeddings(
    model_name = CFG.embeddings_model_repo,
    model_kwargs = {"device": "cuda"}
    # model_kwargs = {"device": "cpu"}
)

# %pip install yandexcloud
# import os
# os.environ["YC_API_KEY"] = "AQVNyWJM7aOhZZ1LXLbI7eNF8J2qmN_V5W-8Wddr"

# from langchain_community.embeddings.yandex import YandexGPTEmbeddings
# embeddings = YandexGPTEmbeddings(YC_API_KEY = 'AQVNyWJM7aOhZZ1LXLbI7eNF8J2qmN_V5W-8Wddr', folder_id='b1gvmo70ll74cvokevfk', model_kwargs = {"device": "cuda"})



# %pip install -q yandex_chain

# from yandex_chain import YandexLLM, YandexEmbeddings

# embeddings = YandexEmbeddings(api_key="AQVNyWJM7aOhZZ1LXLbI7eNF8J2qmN_V5W-8Wddr", folder_id='b1g72uajlds114mlufqi')

.gitattributes:   0%|          | 0.00/1.23k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

train_script.py:   0%|          | 0.00/13.2k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

load INSTRUCTOR_Transformer
max_seq_length  512
CPU times: user 1.31 s, sys: 766 ms, total: 2.07 s
Wall time: 8.54 s


In [ ]:
# truncate =

In [ ]:
# %%time
# FAISS.from_documents(
#     documents = texts[:20], # [:truncate]
#     embedding = embeddings
# )

In [ ]:
# len(texts) / 20 * 90 / 3600

In [ ]:
# for text in texts:
#     if 'Вы можете создать' in text.page_content:
#         print(text)

In [ ]:
### create embeddings and DB
vectordb = FAISS.from_documents(
    documents = texts, # [:truncate]
    embedding = embeddings
)

### persist vector database
# vectordb.save_local(f"{CFG.Output_folder}/faiss_index_hp") # save in output folder
vectordb.save_local(f"{CFG.Embeddings_path}/faiss_index_hp") # save in input folder

In [ ]:
# !zip -r embeddings_full_doc.zip /content/faiss-hp

In [ ]:
# [a.page_content for a in texts[:truncate]]

# Retriever

In [ ]:
def pretty_print_docs(docs):
    print(
        f"\n{'-' * 100}\n".join(
            [f"Document {i+1}:\n\n" + d.page_content for i, d in enumerate(docs)]
        )
    )

In [ ]:
%%time

### download embeddings model
# embeddings = HuggingFaceInstructEmbeddings(
#     model_name = CFG.embeddings_model_repo,
#     model_kwargs = {"device": "cuda"}
# )

# # load vector DB embeddings
# vectordb = FAISS.load_local(
#     CFG.Embeddings_path + '/faiss_index_hp', # from input folder
# #     CFG.Output_folder + '/faiss_index_hp', # from output folder
#     embeddings,
#     allow_dangerous_deserialization=True
# )

clear_output()

CPU times: user 1.96 ms, sys: 0 ns, total: 1.96 ms
Wall time: 2.25 ms


In [ ]:
res = vectordb.similarity_search('Как просмотреть статистику загрузки версий?')
res

In [ ]:
!pip install rank_bm25

In [ ]:
from langchain.retrievers import EnsembleRetriever
from langchain.vectorstores import Chroma
from langchain_community.retrievers import TFIDFRetriever
from langchain_community.retrievers.bm25 import BM25Retriever

# retriever_base =  TFIDFRetriever.from_documents(texts, k=CFG.k)
retriever_base =  BM25Retriever.from_documents(texts, k=CFG.k)
retriever_advanced = vectordb.as_retriever(search_kwargs = {"k": CFG.k, "search_type" : "similarity"})
ensemble_retriever = EnsembleRetriever(retrievers=[retriever_base, retriever_advanced], weights=[0.5, 0.5],k = CFG.k)

In [ ]:
# question = "Как создать аккаунт разработчика?"
# question = 'Как просмотреть статистику загрузки версий?'
question = "Ошибка что значит RuStoreReviewExists?"
# question = "Регистрация разработчиков в RuStore: шаги и опции "
# question = "Как добавить дополнительного разработчика?"
# question = "Как подписаться на топик?"
# question = 'Создать новую учётную запись для разработчика, а затем добавить его в компанию доступным в системе людям'
# question = 'как интегрировать SDK Remote Config для Kotlin'
# question = 'собрать HAR-логи​'
# question = 'Пример отчета по монетизации'
# question = 'Как подключить монетизацию?'
# question = 'Где находится политика конфиденциальности компании?'
# question = 'как использовать VK ID сдк'
similar = ensemble_retriever.invoke(question, k = CFG.k)
# similar = retriever_base.invoke(question, k = CFG.k)
similar

[Document(metadata={'Header 1': '1.0.1', 'Header 2': 'Условия корректной работы SDK'}, page_content='1.0.1 Условия корректной работы SDK Для работы RuStore In-app updates SDK необходимо соблюдение следующих условий.  \n* ОС Android версии 7.0 или выше.\n* Версия RuStore на устройстве пользователя актуальная.\n* Пользователь авторизован в RuStore.\n* Приложению RuStore разрешена установка приложений.'),
 Document(metadata={'Header 1': '1.0.0', 'Header 2': 'Условия корректной работы SDK'}, page_content='1.0.0 Условия корректной работы SDK Для работы RuStore In-app updates SDK необходимо соблюдение следующих условий.  \n* ОС Android версии 7.0 или выше.\n* Версия RuStore на устройстве пользователя актуальная.\n* Пользователь авторизован в RuStore.\n* Приложению RuStore разрешена установка приложений.'),
 Document(metadata={'Header 1': 'Вопросы и ответы'}, page_content='Вопросы и ответы к сведениюНастоящий раздел содержит примеры кода для Kotlin.  \n**Q: Как исправить ошибку «Application i

# False generate

In [ ]:
import requests

query = """
Ты специалист техподдержки российского магазина приложений - RuStore.
Кратко отвечай на предоставленный вопрос от клиента - разработчика приложений.
"""


def yagpt_request(question, context):
    url = "https://llm.api.cloud.yandex.net/foundationModels/v1/completion"
    headers = {
        "Content-Type": "application/json",
        "Authorization": "Api-Key AQVNyWJM7aOhZZ1LXLbI7eNF8J2qmN_V5W-8Wddr"
    }

    ya_prompt = {
        "modelUri": "gpt://b1g72uajlds114mlufqi/yandexgpt/latest",
        "completionOptions": {
            "stream": False,
            "temperature": 0.0,
            "maxTokens": "1024"
        },
        "messages": [
            {
                "role": "system",
                "text": context
            },
            {
                "role": "user", "text": question
            }
        ]
    }

    response = requests.post(url, headers=headers, json=ya_prompt)
    result = eval(response.text)

    return result['result']['alternatives'][0]['message']['text']


def refactor_sim_search_res(question):

    context = query

    text = yagpt_request(question, context)
    return text

In [ ]:
question = "Как добавить дополнительного разработчика?"

false_answer = refactor_sim_search_res(question)

retriever_out = ensemble_retriever.invoke(false_answer, k = CFG.k)

## Evaluate

In [ ]:
import pandas as pd
questions = pd.read_json('questions_for_model.json')

for question in questions.index:

    false_answer = refactor_sim_search_res(question)

    similar = ensemble_retriever.invoke(false_answer, k = 1)[0]
    print(question, '--->', false_answer, '\n'+ '*' * 10 + '\n', similar.page_content, '\n' + '-'*50)

In [ ]:
import pandas as pd
questions = pd.read_json('questions_for_model.json')

for question in questions.index:
    similar = retriever_advanced.invoke(question, k = 1)[0]
    print(question, '\n', similar.page_content, '\n' + '-'*50)

In [ ]:
import pandas as pd
questions_orgs = pd.read_excel('Пример вопросов с ответами с испольхованием хэлпы.xlsx')

for question in questions_orgs['Вопрос']:
    similar = ensemble_retriever.invoke(question, k = 1)[0]
    print(question, '\n', similar.page_content, '\n' + '-'*50)

# Model

In [ ]:
1/0

In [ ]:
import requests

query = """
Чтобы ответить на вопрос, используй только следующие фрагменты контекста.
Если ответа нет в контексте скажи об этом.
Ответ должен быть в markdown, сохрани изначальную структуру контекста, и отобрази максимально полный ответ.

Контекст:
{context}
"""


def yagpt_request(question, context):
    url = "https://llm.api.cloud.yandex.net/foundationModels/v1/completion"
    headers = {
        "Content-Type": "application/json",
        "Authorization": "Api-Key AQVNyWJM7aOhZZ1LXLbI7eNF8J2qmN_V5W-8Wddr"
    }

    ya_prompt = {
        "modelUri": "gpt://b1g72uajlds114mlufqi/yandexgpt/latest",
        "completionOptions": {
            "stream": False,
            "temperature": 0.0,
            "maxTokens": "1024"
        },
        "messages": [
            {
                "role": "system",
                "text": context
            },
            {
                "role": "user", "text": question
            }
        ]
    }

    response = requests.post(url, headers=headers, json=ya_prompt)
    result = eval(response.text)

    return result['result']['alternatives'][0]['message']['text']


def refactor_sim_search_res(question):
    retriever_ans_raw = ensemble_retriever.invoke(question, k=6)
    retriever_ans = '\n\n'.join([i.page_content for i in retriever_ans_raw])

    context = query

    text = yagpt_request(question, context)
    return text, retriever_ans_raw

In [ ]:
question = 'как интегрировать SDK Remote Config для Kotlin'

In [ ]:
refactor_sim_search_res(question)

In [ ]:
ans, retriever_ans_raw = refactor_sim_search_res(question)
print(ans)

Чтобы интегрировать SDK Remote Config для Kotlin, ознакомьтесь с примером реализации — приложением-примером на сайте gitflic.ru. Это поможет правильно интегрировать SDK Remote Config.

Также можно изучить документацию по работе с параметрами в RuStore Remote Config и настройке работы с ними.


In [ ]:
retriever_ans_raw

In [ ]:
context

# Paraphrase

In [ ]:
# query_paraphrase = """
# Ты помогаешь создать ИИ-помощника по документации RuSrore.
# Ты получаешь на вход вопрос пользователя, который нужно переформулировать, чтобы по нему было легко найти нужную информацию в документации.
# """
# query_paraphrase = """
# Ты помогаешь создать ИИ-помощника по документации RuSrore.
# Ты получаешь на вход вопрос пользователя, который нужно переформулировать, чтобы получится текст, с помощью которого можно будет быстро найти нужную информацию в документации по ключевым словам.
# """
# query_paraphrase = """
# Ты помогаешь создать ИИ-помощника по документации RuSrore.
# На вход получаешь вопрос пользователя, извлеки из него ключевую информацию, чтобы можно было по ключевым словам найти нужную информацию в докуменатации
# """
# query_paraphrase = """
# Ты помогаешь создать ИИ-помощника по документации RuSrore.
# Перефразируй вопрос пользователя так, чтобы нужную информацию можно было легко найти по ключевым словам в документации.
# Пример: вопрос 'Можно ли и откуда доставать данные гендера и возраста человека, скачавшего приложение? Необходимо для более точной настройки таргета РСЯ' переформулируем в 'Данные гендера и возраста человека, скачавшего приложение'
# Напиши только переформулированный вопрос
# """
query_paraphrase = """
Ты помогаешь создать ИИ-помощника по документации RuSrore.
Перефразируй вопрос пользователя, удалив ненужную информацию
Пример: вопрос 'Можно ли и откуда доставать данные гендера и возраста человека, скачавшего приложение? Необходимо для более точной настройки таргета РСЯ' переформулируем в 'Получение данных гендера и возраста человека, скачавшего приложение'
Напиши только переформулированный вопрос
"""

def yagpt_rparaphrase(question):
    url = "https://llm.api.cloud.yandex.net/foundationModels/v1/completion"
    headers = {
        "Content-Type": "application/json",
        "Authorization": "Api-Key AQVNyWJM7aOhZZ1LXLbI7eNF8J2qmN_V5W-8Wddr"
    }

    ya_prompt = {
        "modelUri": "gpt://b1g72uajlds114mlufqi/yandexgpt/latest",
        "completionOptions": {
            "stream": False,
            "temperature": 0.0,
            "maxTokens": "1024"
        },
        "messages": [
            {
                "role": "system",
                "text": query_paraphrase
            },
            {
                "role": "user", "text": question
            }
        ]
    }

    response = requests.post(url, headers=headers, json=ya_prompt)
    result = eval(response.text)

    return result['result']['alternatives'][0]['message']['text']

In [ ]:
question = 'как интегрировать SDK Remote Config для Kotlin'
ans = yagpt_rparaphrase(question)
ans

'Интеграция SDK Remote Config для Kotlin.'

## Evaluate paraphrase

In [ ]:
questions_orgs = pd.read_excel('/content/Пример вопросов с ответами с испольхованием хэлпы.xlsx')

In [ ]:
for question in questions_orgs['Вопрос']:
    question_paraphrased = yagpt_rparaphrase(question)
    similar = ensemble_retriever.invoke(question_paraphrased, k = 1)[0]
    print(question, '--->', question_paraphrased, '\n', similar.page_content, '\n' + '-'*50)

In [ ]:
for question in questions_orgs['Вопрос']:
    question_paraphrased = yagpt_rparaphrase(question)
    similar = ensemble_retriever.invoke(question_paraphrased, k = 1)[0]
    print(question, '--->', question_paraphrased, '\n', similar.page_content, '\n' + '-'*50)
    refactor_sim_search_res(question)

In [ ]:
for question in questions.index:
    question_paraphrased = yagpt_rparaphrase(question)
    retriever_ans = ensemble_retriever.invoke(question_paraphrased, k = 1)[:4]

    retriever_ans = '\n\n'.join([i.page_content for i in retriever_ans])
    context = query.format(context=retriever_ans)
    text = yagpt_request(question_paraphrased, context)


    print(question, '--->', question_paraphrased, '\n', text, '\n' + '-'*50)